In [55]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt

model = pybamm.BaseModel()

In [56]:
phi = pybamm.Variable("Positive electrode potential [V]", domain="positive electrode")

In [57]:
phi_e_s = pybamm.Variable("Separator electrolyte potential [V]", domain="separator")
phi_e_p = pybamm.Variable("Positive electrolyte potential [V]", domain="positive electrode")

In [58]:
phi_e = pybamm.concatenation(phi_e_s, phi_e_p)

In [59]:
c = pybamm.Variable(
    "Positive particle concentration [mol.m-3]",
    domain="positive particle",
    auxiliary_domains={
        "secondary": "positive electrode",
    },
)

In [60]:
from scipy import constants

F = pybamm.Parameter("Faraday constant [C.mol-1]")
R = pybamm.Parameter("Molar gas constant [J.mol-1.K-1]")
T = pybamm.Parameter("Temperature [K]")

a = pybamm.Parameter("Surface area per unit volume [m-1]")
R_p = pybamm.Parameter("Positive particle radius [m]")
L_s = pybamm.Parameter("Separator thickness [m]")
L_p = pybamm.Parameter("Positive electrode thickness [m]")
A = pybamm.Parameter("Electrode cross-sectional area [m2]")

sigma = pybamm.Parameter("Positive electrode conductivity [S.m-1]")
kappa = pybamm.Parameter("Electrolyte conductivity [S.m-1]")
D = pybamm.Parameter("Diffusion coefficient [m2.s-1]")

I_app = pybamm.Parameter("Applied current [A]")
c0 = pybamm.Parameter("Initial concentration [mol.m-3]")

In [61]:
c_surf = pybamm.surf(c)  # get the surface concentration
inputs = {"Positive particle surface concentration [mol.m-3]": c_surf}
j0 =  pybamm.FunctionParameter("Positive electrode exchange-current density [A.m-2]", inputs)
U = pybamm.FunctionParameter("Positive electrode OCP [V]", inputs)

In [62]:

j_s = pybamm.PrimaryBroadcast(0, "separator")
j_p =  2 * j0 * pybamm.sinh((F / 2 / R / T) * (phi - phi_e_p - U))
j = pybamm.concatenation(j_s, j_p)

In [63]:
# charge conservation equations 
i = -sigma * pybamm.grad(phi)
i_e = -kappa * pybamm.grad(phi_e)
model.algebraic = {
    phi: pybamm.div(i) + a * j_p,
    phi_e: pybamm.div(i_e) - a * j,
}
# particle equations (mass conservation)
N = -D * pybamm.grad(c)  # flux
dcdt = -pybamm.div(N)
model.rhs = {c: dcdt}  

# boundary conditions 
model.boundary_conditions = {
    phi: {"left": (pybamm.Scalar(0), "Neumann"), "right": (-I_app / A / sigma, "Neumann")},
    phi_e: {"left": (pybamm.Scalar(0), "Dirichlet"), "right": (pybamm.Scalar(0), "Neumann")},
    c: {"left": (pybamm.Scalar(0), "Neumann"), "right": (-j_p / F / D, "Neumann")}
}

# initial conditions
inputs = {"Initial concentration [mol.m-3]": c0}
U_init = pybamm.FunctionParameter("Positive electrode OCP [V]", inputs)
model.initial_conditions = {
    phi: U_init,
    phi_e: 0,
    c: c0
}

In [64]:
model.variables = {
    "Positive electrode potential [V]": phi,
    "Electrolyte potential [V]": phi_e,
    "Positive particle concentration [mol.m-3]": c,
    "Positive particle surface concentration [mol.m-3]": c_surf,
    "Average positive particle surface concentration [mol.m-3]": pybamm.x_average(c_surf),
    "Positive electrode interfacial current density [A.m-2]": j_p,
    "Positive electrode OCP [V]":pybamm.boundary_value(U, "right"),
    "Terminal voltage [V]": pybamm.boundary_value(phi, "right"),
}

In [65]:
from pybamm import tanh

# both functions will depend on the maximum concentration 
c_max = pybamm.Parameter("Maximum concentration in positive electrode [mol.m-3]")


def exchange_current_density(c_surf):
    k = 6 * 10 ** (-7)   # reaction rate [(A/m2)(m3/mol)**1.5]
    c_e = 1000  # (constant) electrolyte concentration [mol.m-3]
    return k * c_e** 0.5 * c_surf ** 0.5 * (c_max - c_surf) ** 0.5

def open_circuit_potential(c_surf):
    stretch = 1.062
    sto = stretch * c_surf / c_max
    
    u_eq = (
        2.16216
        + 0.07645 * tanh(30.834 - 54.4806 * sto)
        + 2.1581 * tanh(52.294 - 50.294 * sto)
        - 0.14169 * tanh(11.0923 - 19.8543 * sto)
        + 0.2051 * tanh(1.4684 - 5.4888 * sto)
        + 0.2531 * tanh((-sto + 0.56478) / 0.1316)
        - 0.02167 * tanh((sto - 0.525) / 0.006)
    )
    return u_eq

In [66]:
param = pybamm.ParameterValues(
    {
        "Surface area per unit volume [m-1]":0.15e6,
        "Positive particle radius [m]": 10e-6,
        "Separator thickness [m]": 25e-6,
        "Positive electrode thickness [m]": 100e-6,
        "Electrode cross-sectional area [m2]": 2.8e-2,
        "Applied current [A]": 0.9,
        "Positive electrode conductivity [S.m-1]": 10,
        "Electrolyte conductivity [S.m-1]": 1,
        "Diffusion coefficient [m2.s-1]": 1e-13,
        "Faraday constant [C.mol-1]": 96485,
        "Initial concentration [mol.m-3]": 25370,
        "Molar gas constant [J.mol-1.K-1]": 8.314,
        "Temperature [K]": 298.15,
        "Maximum concentration in positive electrode [mol.m-3]": 51217,
        "Positive electrode exchange-current density [A.m-2]": exchange_current_density,
        "Positive electrode OCP [V]": open_circuit_potential,
    }
)

In [67]:
r = pybamm.SpatialVariable(
    "r", 
    domain=["positive particle"],     
    auxiliary_domains={
        "secondary": "positive electrode"
    },
    coord_sys="spherical polar")
x_s = pybamm.SpatialVariable("x_s", domain=["separator"], coord_sys="cartesian")
x_p = pybamm.SpatialVariable("x_p", domain=["positive electrode"], coord_sys="cartesian")


geometry = {
    "separator": {x_s: {"min": -L_s, "max": 0}},        
    "positive electrode": {x_p: {"min": 0, "max": L_p}},
    "positive particle": {r: {"min": 0, "max": R_p}},
}

In [68]:
param.process_model(model)
param.process_geometry(geometry)

In [69]:
submesh_types = {
    "separator": pybamm.Uniform1DSubMesh,
    "positive electrode": pybamm.Uniform1DSubMesh,
    "positive particle": pybamm.Uniform1DSubMesh,
}
var_pts = {x_s: 10, x_p: 20, r: 30}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)

spatial_methods = {
    "separator": pybamm.FiniteVolume(),
    "positive electrode": pybamm.FiniteVolume(),
    "positive particle": pybamm.FiniteVolume(),
}
disc = pybamm.Discretisation(mesh, spatial_methods)
disc.process_model(model);

In [95]:
# solve
solver = pybamm.CasadiSolver(root_tol=1e-3)
t_eval = np.linspace(0, 10, 101)
sim = solver.solve(model, t_eval)

In [96]:
# plot
pybamm.dynamic_plot(
    sim,
    [
        "Positive electrode potential [V]",
        "Electrolyte potential [V]",
        "Positive electrode interfacial current density [A.m-2]",
        "Positive particle surface concentration [mol.m-3]",
        "Average positive particle surface concentration [mol.m-3]",
        ["Positive electrode OCP [V]", "Terminal voltage [V]"],
    ],
)

2022-12-14 18:05:26.332 - [WARNING] processed_variable.get_spatial_scale(518): No length scale set for positive electrode. Using default of 1 [m].
2022-12-14 18:05:26.347 - [WARNING] processed_variable.get_spatial_scale(518): No length scale set for separator. Using default of 1 [m].
2022-12-14 18:05:26.355 - [WARNING] processed_variable.get_spatial_scale(518): No length scale set for positive electrode. Using default of 1 [m].
2022-12-14 18:05:26.372 - [WARNING] processed_variable.get_spatial_scale(518): No length scale set for positive electrode. Using default of 1 [m].


interactive(children=(FloatSlider(value=0.0, description='t', max=10.0), Output()), _dom_classes=('widget-inte…

In [97]:
model.variables.search("")

Average positive particle surface concentration [mol.m-3]
Electrolyte potential [V]
Positive electrode OCP [V]
Positive electrode interfacial current density [A.m-2]
Positive electrode potential [V]
Positive particle concentration [mol.m-3]
Positive particle surface concentration [mol.m-3]
Terminal voltage [V]


In [98]:
pybamm.dynamic_plot(
    sim,
    [
        "Positive particle surface concentration [mol.m-3]","Positive particle concentration [mol.m-3]",
    ],
)

2022-12-14 18:05:29.212 - [WARNING] processed_variable.get_spatial_scale(518): No length scale set for positive particle. Using default of 1 [m].
2022-12-14 18:05:29.213 - [WARNING] processed_variable.get_spatial_scale(518): No length scale set for positive electrode. Using default of 1 [m].


interactive(children=(FloatSlider(value=0.0, description='t', max=10.0), Output()), _dom_classes=('widget-inte…

In [102]:
c_surf

BoundaryValue(0x27d4e531e2a49b31, boundary value, children=['Positive particle concentration [mol.m-3]'], domains={'primary': ['positive electrode']})